## Importing module

In [1]:
import numpy as np

## Statistical model

In [2]:
import os
os.chdir('/Users/nicopalm/Documents/Projects/ode/')

In [3]:
from src.minimizer.minimizer_library.differential_evolution import DifferentialEvolution
from src.parametric_function_library.aging_model_Naumann import AgingModelNaumann
from src.statistical_models.statistical_model_library.gaussian_noise_model import GaussianNoiseModel

In [4]:
theta = np.array([4, 2300, 0.8])
sigma = 0.029 ** 2

lower_bounds_x = np.array([0.1, 279.15])
upper_bounds_x = np.array([1, 333.15])

lower_bounds_theta = np.array([0.1, 0.001, 0.1])
upper_bounds_theta = np.array([10, 10000, 1])

In [5]:
parametric_function = AgingModelNaumann()

In [6]:
statistical_model = GaussianNoiseModel(function=parametric_function, lower_bounds_x=lower_bounds_x,
                                       upper_bounds_x=upper_bounds_x, lower_bounds_theta=lower_bounds_theta,
                                       upper_bounds_theta=upper_bounds_theta, sigma=sigma)

## Minimizer

In [7]:
minimizer = DifferentialEvolution()

## Blackbox function

In [8]:
def blackbox_model(x):
    return statistical_model.random(theta=theta, x=x)

## Designs

In [9]:
# Designs
from src.benchmarking.benchmarking import Benchmarking
from src.designs_of_experiments.design_library.d_design import DDesign
from src.designs_of_experiments.design_library.latin_hypercube import LatinHypercube
from src.designs_of_experiments.design_library.pi_design import PiDesign
from src.designs_of_experiments.design_library.random import Random

In [10]:
number_designs = 6
number_of_evaluations = 1000

### Latin Hypercube

In [11]:
LH = LatinHypercube(lower_bounds_design=lower_bounds_x, upper_bounds_design=upper_bounds_x,
                    number_designs=number_designs)

### Random

In [12]:
random_design = Random(number_designs=number_designs, lower_bounds_design=lower_bounds_x,
                       upper_bounds_design=upper_bounds_x)

### Pi-design

In [13]:
initial_theta = statistical_model.calculate_maximum_likelihood_estimation(
    x0=LH.design, y=np.array([blackbox_model(x) for x in LH.design]), minimizer=minimizer)

print("The estimated theta is \n",initial_theta)
print("The real theta is \n",theta)

The estimated theta is 
 [9.93363267e+00 2.67648560e+03 9.87861800e-01]
The real theta is 
 [4.0e+00 2.3e+03 8.0e-01]


In [14]:
print('The cramer rao lower bound at the estimated theta is \n',
      statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta))
print('The relative expected standard deviations of the parameter estimators are \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta).diagonal())/initial_theta)
parameter = int(input('Which parameter should be minimized? \n'))

The cramer rao lower bound at the estimated theta is 
 [[1.94068670e+03 3.18253131e+04 1.54072708e+00]
 [3.18253131e+04 1.11688370e+06 3.05744692e+02]
 [1.54072708e+00 3.05744692e+02 1.95556382e-01]]
The relative expected standard deviations of the parameter estimators are 
 [4.43475486 0.39485629 0.44765125]
Which parameter should be minimized? 
0


In [15]:
%%time
min_entry = PiDesign(number_designs=number_designs, 
                     lower_bounds_design=lower_bounds_x,
                     upper_bounds_design=upper_bounds_x,
                     column=parameter, 
                     row=parameter, 
                     initial_theta=theta,
                     statistical_model=statistical_model, 
                     previous_design=LH,
                     minimizer=minimizer)

Calculating the Parameter individual design...

finished!
CPU times: user 17.8 s, sys: 8.67 s, total: 26.5 s
Wall time: 9.28 s


### Maximum determinant of Fisher Information matrix

In [16]:
%%time
max_det = DDesign(number_designs=number_designs, lower_bounds_design=lower_bounds_x,
                  upper_bounds_design=upper_bounds_x, initial_theta=theta,
                  statistical_model=statistical_model, minimizer=minimizer)

Calculating the Maximum determinant design...

finished!
CPU times: user 3.98 s, sys: 256 ms, total: 4.23 s
Wall time: 3.77 s


In [17]:
designs_of_experiments=[LH, random_design, min_entry, max_det]

### Plotting

In our case, we can plot the design points since they lie in the two dimensional real plane.

In [18]:
# plot design points
from src.visualization.plotting_functions import *
from ipywidgets import interact

data = [dot_scatter(x_dots=designs_of_experiments[0].design.T[0],
                    y_dots=designs_of_experiments[0].design.T[1],fill=None)]
fig0 = go.FigureWidget(data=data)
fig0 = update_layout_of_graph(fig0,
                              title='Design points')

@interact(index=range(len(designs_of_experiments)))
def update(index='LH'):
    with fig0.batch_update():
        fig0.update_layout(title=designs_of_experiments[index].name)
        fig0.data[0].x = designs_of_experiments[index].design.T[0]
        fig0.data[0].y = designs_of_experiments[index].design.T[1]
        
    return fig0

interactive(children=(Dropdown(description='index', options=(0, 1, 2, 3), value=0), Output()), _dom_classes=('…

## Benchmarking

In [19]:
benchmarking = Benchmarking(blackbox_model=blackbox_model, statistical_model=statistical_model,
                            designs_of_experiments=designs_of_experiments)

In [ ]:
benchmarking.evaluate_designs(number_of_evaluations=number_of_evaluations, minimizer=minimizer)

Evaluate the Random design: 100%|███████████| 1000/1000 [02:41<00:00,  6.19it/s]
Evaluate the Parameter individual design: 100%|█| 1000/1000 [04:48<00:00,  3.46i
Evaluate the Maximum determinant design:  19%|▏| 186/1000 [00:29<02:09,  6.28it/

In [ ]:
benchmarking.plot_estimations()

## Metrics

In [ ]:
from src.metrics.metric_library.determinant_of_fisher_information_matrix import DeterminantOfFisherInformationMatrix
from src.metrics.metric_library.estimation_mean_error import EstimationMeanError
from src.metrics.metric_library.estimation_mean_parameter_estimations import EstimationMeanParameterEstimations
from src.metrics.metric_library.estimation_variance_parameter_estimations import EstimationVarianceParameterEstimations
from src.metrics.metric_library.k_fold_cross_validation import KFoldCrossValidation

### Determinant of Fisher information matrix

In [ ]:
det_metric = DeterminantOfFisherInformationMatrix(theta=theta, statistical_model=statistical_model)

In [ ]:
fig = det_metric.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline="max",
)
fig.show()

### Estimated mean of parameter estimation

In [ ]:
estimation_mean_parameter_estimation=EstimationMeanParameterEstimations()

In [ ]:
estimation_mean_parameter_estimation.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline=theta,
)


### Estimated variance of parameter estimation

In [ ]:
estimation_var_parameter_estimation=EstimationVarianceParameterEstimations()

In [ ]:
baseline = np.array([statistical_model.calculate_cramer_rao_lower_bound(
    x0=design.design, theta=theta).diagonal() for design in designs_of_experiments]).T
baseline

In [ ]:
estimation_var_parameter_estimation.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline=baseline,
)

### Estimated mean error

This error function allows us to estimate the mean error in the model for the error in the estimating the correct parameter.

In [ ]:
estimation_mean_error = EstimationMeanError(number_evaluations=100, theta=theta,
                               statistical_model=statistical_model)

In [ ]:
estimation_mean_error.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline="min",
)

## Validation

### K-fold cross validation

In [ ]:
k_fold_data = {}
for design in benchmarking.evaluations_blackbox_function.keys():
    k_fold_data[design] = benchmarking.evaluations_blackbox_function[design][0]

In [ ]:
k_fold_validation = KFoldCrossValidation(statistical_model=statistical_model, minimizer=minimizer)

In [ ]:
k_fold_validation.plot(
    evaluations_blackbox_function_for_each_design=k_fold_data,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline="min",
)